In [2]:
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (10,10)
mpl.rcParams['axes.grid'] = False
import re
import numpy as np
from PIL import Image
import time
import functools
import os
import pandas as pd
import librosa
import glob
import tensorflow as tf
import tensorflow.contrib.eager as tfe
import librosa.display
import hashlib

In [3]:
from os import listdir
from os.path import isfile, join
from os import walk

import glob

In [4]:
labels = listdir('C:/Users/karth/Documents/Speech data')

In [6]:
from sklearn.preprocessing import LabelEncoder

In [7]:
labels = np.array(labels)

In [8]:
validationlist = open('C:/Users/karth/Documents/Speech data txt/validation_list.txt')
testinglist = open('C:/Users/karth/Documents/Speech data txt/testing_list.txt')

In [9]:
validation_files = validationlist.read().splitlines()
testing_files = testinglist.read().splitlines()
X_validation = []
Y_validation = []
X_testing = []
Y_testing = []
X_training = []
Y_training = []

In [21]:
for file in validation_files:
    data, sampling_rate = librosa.load('C:/Users/karth/Documents/Speech data/' + file) #replace with location of files
    data = np.pad(data, (0,22050-len(data)), mode='constant', constant_values=0)
    mfccs = librosa.feature.mfcc(y=data, sr=sampling_rate, n_mfcc=20)
    X_validation.append(mfccs)
    label = file.partition('/')[0]
    Y_validation.append(label)
for file in testing_files:
    data, sampling_rate = librosa.load('C:/Users/karth/Documents/Speech data/' + file)
    data = np.pad(data, (0,22050-len(data)), mode='constant', constant_values=0)
    mfccs = librosa.feature.mfcc(y=data, sr=sampling_rate, n_mfcc=20) 
    X_testing.append(mfccs)
    label = file.partition('/')[0]
    Y_testing.append(label)

In [22]:
non_training_files = validation_files + testing_files

In [23]:
for label in labels:
    dir = 'C:/Users/karth/Documents/Speech data/' + label
    labelfiles = listdir(dir)
    for i in labelfiles:
        file = label + '/' + i
        if file in non_training_files:
            continue
        else:
            data, sampling_rate = librosa.load('C:/Users/karth/Documents/Speech data/' + file)
            data = np.pad(data, (0,22050-len(data)), mode='constant', constant_values=0)
            mfccs = librosa.feature.mfcc(y=data, sr=sampling_rate, n_mfcc=20) 
            X_training.append(mfccs)
            Y_training.append(label)

In [24]:
X_np_training = np.array(X_training)
Y_np_training = np.array(Y_training)
X_np_validation = np.array(X_validation)
Y_np_validation = np.array(Y_validation)
X_np_testing = np.array(X_testing)
Y_np_testing = np.array(Y_testing)

In [25]:
#np.save('X_training.npy', X_np_training) - remove comment when running for the first time and creating the numpy sets
#np.save('Y_training.npy', Y_np_training) 
#np.save('X_validation.npy', X_np_validation) 
#np.save('Y_validation.npy', Y_np_validation)
#np.save('X_testing.npy', X_np_testing)
#np.save('Y_testing.npy', Y_np_testing)


In [10]:
from keras.callbacks import ModelCheckpoint
from keras.models import Model, load_model, Sequential
from keras.layers import Dense, Activation, Dropout, Input, Masking, TimeDistributed, LSTM, Conv1D
from keras.layers import GRU, Bidirectional, BatchNormalization, Reshape
from keras.optimizers import Adam

Using TensorFlow backend.


In [1]:
import keras
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

temp1 = labelencoder.fit_transform(Y_np_training)
Y2_training = [x if x < 5 else 5 for x in temp1]
Y_oh_training = keras.utils.to_categorical(Y2_training) #using 5 of the trigger words to be recognized and the rest in one label category

temp2 = labelencoder.fit_transform(Y_np_testing)
Y2_testing = [x if x < 5 else 5 for x in temp2]
Y_oh_testing = keras.utils.to_categorical(Y2_testing)

temp3 = labelencoder.fit_transform(Y_np_validation)
Y2_validation = [x if x < 5 else 5 for x in temp3]

Y_oh_validation = keras.utils.to_categorical(Y2_validation)

ModuleNotFoundError: No module named 'keras'

In [95]:
#np.save('Y_oh_testing.npy', Y_oh_testing)
#np.save('Y_oh_validation.npy', Y_oh_validation)
#np.save('Y_oh_training.npy', Y_oh_training) 

In [11]:
X_np_training = np.load('X_training.npy')
Y_np_training = np.load('Y_training.npy') 
X_np_validation = np.load('X_validation.npy') 
Y_np_validation = np.load('Y_validation.npy')
X_np_testing = np.load('X_testing.npy')
Y_np_testing = np.load('Y_testing.npy')
Y_oh_testing = np.load('Y_oh_testing.npy')
Y_oh_validation = np.load('Y_oh_validation.npy')
Y_oh_training = np.load('Y_oh_training.npy') 

In [103]:
num_labels = Y_oh_training.shape[1]
model2 = Sequential()
model2.add(LSTM(32, input_shape = (20,44)))
model2.add(Activation('relu'))
model2.add(Dense(num_labels))
model2.add(Activation('softmax'))

In [104]:
model2.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
print (model2.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 32)                9856      
_________________________________________________________________
activation_9 (Activation)    (None, 32)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 6)                 198       
_________________________________________________________________
activation_10 (Activation)   (None, 6)                 0         
Total params: 10,054
Trainable params: 10,054
Non-trainable params: 0
_________________________________________________________________
None


In [105]:
model2.fit
(X_np_training, Y_oh_training, batch_size=32, epochs=100, validation_data=(X_np_validation, Y_oh_validation))

Train on 84843 samples, validate on 9981 samples
Epoch 1/5
84843/84843 [==============================] - 345s 4ms/step - loss: 0.4902 - acc: 0.9011 - val_loss: 0.4595 - val_acc: 0.9073
Epoch 2/5
84843/84843 [==============================] - 329s 4ms/step - loss: 0.4593 - acc: 0.9056 - val_loss: 0.4575 - val_acc: 0.9073
Epoch 3/5
84843/84843 [==============================] - 327s 4ms/step - loss: 0.4546 - acc: 0.9056 - val_loss: 0.4577 - val_acc: 0.9073
Epoch 4/5
84843/84843 [==============================] - 326s 4ms/step - loss: 0.4504 - acc: 0.9056 - val_loss: 0.4538 - val_acc: 0.9073
Epoch 5/5
84843/84843 [==============================] - 326s 4ms/step - loss: 0.4467 - acc: 0.9056 - val_loss: 0.4550 - val_acc: 0.9073


In [107]:
from keras.models import load_model

model.save('my_model1.h5')  